# Natural Language Processing(4): Topic Modeling

Another popular text analysis technique is **Topic Modeling**. The goal of topic modeling is to find various topics that are present in corpus.

One of the topic modeling techniques is called **Latent Dirichlet Allocation (LDA)**(Latent is for hidden, Dirichlet is for type of probability distribution).(Other topic modeling: Non-Negative Matrix Factorization(NMF) and Latent Semantic Indexing(LSI)). To use topi modeling technique, we have to provide (1) a document term matrix(DTM)(since order doesn't matter here); (2) the numbers of topics we would like the algorithm to pick up.

If the results don't make sense, we have to change up the number of topics, the terms in the DTM, or even try a different model.

## Topic Modeling - Attemp # 1(All Text)

In [ ]:
import pandas as pd
import pickle

data = pd.read_pickle('dtm_stop.pkl')
data

In [ ]:
# One of the required inputs is document term matrix
tdm = data.transpose()
tdm.head()

In [ ]:
# Import necessary modeules for LDA with gensim
from genism import matutils, models
from scipy.sparse import csr_matrix

sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [ ]:
# Genism also requires dictionary of all terms and their respective location in TDM
cv = pickle.load(open('cv_stop.pkl', 'rb'))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

Now that we have the document term matrix and id2word(dictionary of location: term), we need to specify two other parameters - the number of topics and the number of passes. We can start the number of topics at random, if the results don't make sense, we can increase the number from there.

In [ ]:
lda = models.LdaModel(corpus = corpus, id2word = id2word, num_topics = 2, passes = 10)
lda.print_topics()

## Topic Modeling - Attempt #2(Nouns Only)

One popular trick is to look at terms that are from one part of speech(only nouns, only adjectives, etc).

In [ ]:
# Create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nous(text):
    # Given a string of text, tokenize the text and pull out only the nouns
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouss = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)]
    return " ".join(all_nouns)

In [ ]:
data_clean = pd.read_pickle('data_clean.pkl')
data_clean

In [ ]:
data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
data_nouns

In [ ]:
# Create a new document-term matrix only using nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words = stop_words)
data_cvn = cvn.fit_transform(data_nouns.transcript)
data_dtmn = pd.DataFrame(data_cvn.toarry(), columns = cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

In [ ]:
# Create the genism corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocablulary dicionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [ ]:
ldan = models.LdaModel(corpus = corpusn, num_topics = 2, id2word = id2wordn, passes = 10)
ldan.print_topics()

## Topic Modeling - Attempt #3 (Nouns and Adjectives)

In [ ]:
# Create a function to pull out nouns and adjectives from a string of text
def nouns_adj(text):
    is_nous_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nous_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)]
    return " ".join(nouns_adj)

In [ ]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

In [ ]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

In [ ]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [ ]:
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()